# Planning 

### their will be folllowing steps 
    1. predict name from classifier5
    2. predict name and email from old-rp
    3. decide final name and email

In [1]:
from keras.models import load_model
from VectorProcessing import GramGloveSentenceVector

Using TensorFlow backend.


In [2]:
classifier_path = "/home/sumit/Desktop/project17/ResumeParser/Data/resume_segments/Test/classifier5"
dir_path = "/home/sumit/Desktop/project17/ResumeParser/Data/resume_samples/txts/samples/"
import os
dir_list = os.listdir(dir_path)
print dir_list

['MichellePegler.pdf.txt', 'InnaBarmash.pdf.txt', 'JacobStelman.pdf.txt', 'TT HaswinyDeva.pdf.txt', 'StevenDang.pdf.txt', 'TylerDaluz.pdf.txt', 'DanielMintzLinkedIn.pdf.txt', 'EdwinAmirian.pdf.txt', 'StevenAguiar.pdf.txt', 'BrendanWhiting.pdf.txt', 'TanzimMilkey.pdf.txt', 'VeronicaShea.pdf.txt', 'MichelleGeng.pdf.txt', 'PavelGurevich.pdf.txt', 'MasonBrunnick.pdf.txt', 'AshleyLyle.pdf.txt', 'ErinPellegrino.pdf.txt', 'RosalynLau.pdf.txt', 'TTHaswinyDeva.pdf.txt', 'KatherineErnst.pdf.txt', 'SophiaFeng.pdf.txt', 'AndrewAng.pdf.txt', 'TrippPettigrew-Rolapp.pdf.txt', 'MicahTillman.pdf.txt', 'LeahKovach.pdf.txt', 'OriRatner.pdf.txt', 'IsaacMast2.pdf.txt', 'Shilpika (Pi)Chowdhury.pdf.txt', 'AbbyTMiller.pdf.txt', 'ThomasWasko.pdf.txt', 'TrishaHaut\xc3\xa9a.pdf.txt', 'Shilpika(Pi)Chowdhury.pdf.txt', 'IanLee.pdf.txt', 'JohannChristineAlcaraz.pdf.txt', 'SamStoner.pdf.txt', 'RahulNemani.pdf.txt', 'JonnyLeaton.pdf.txt', 'Emilie LimaBurke.pdf.txt', 'NikeOnifade.pdf.txt', 'PatrickS.Lee,PMP.pdf.txt', '

In [3]:
gsv = GramGloveSentenceVector(dir_path+dir_list[0],dimension=300, training=False)
sen2vec = gsv.get_5gram_sentenceVector()
print len(sen2vec), len(sen2vec[0])

43 1500


In [4]:
classifier = load_model(classifier_path)

In [5]:
import numpy as np
y_pred = classifier.predict(np.array(sen2vec))

In [11]:
with open(dir_path+dir_list[0], 'r') as f:
    content = f.readlines()
count = 0
labels = ['basic', 'experience', 'education', 'certificate', 'extra', 'skills', 'projects','summary', 'mimc']
basic = []
for line in content:
    if np.argmax(y_pred, 1)[count] == 0:
        basic.append(line.strip())
    count += 1
print basic

['Michelle Pegler', 'Senior Business Intelligence Analyst, Federal Reserve', 'Smartly', 'Smartly', '', 'Page1', '', 'Michelle Pegler', '']


In [9]:
from nltk.tag import StanfordNERTagger
st = StanfordNERTagger('/home/sumit/stanford/stanford-ner/classifiers/english.muc.7class.distsim.crf.ser.gz', '/home/sumit/stanford/stanford-ner/stanford-ner.jar')

def getNER(data):
    return st.tag(data.split())

In [16]:
ner_basic = []
for line in basic:
    ner_basic.append(getNER(line))
print ner_basic
name = []
temp = []
for i in range(len(ner_basic)):
    for j in range(len(ner_basic[i])):
        if ner_basic[i][j][1] == 'PERSON':
            temp.append(ner_basic[i][j][0])
    if len(temp):
        name.append(" ".join(temp))
    temp = []
print name

[[(u'Michelle', u'PERSON'), (u'Pegler', u'PERSON')], [(u'Senior', u'O'), (u'Business', u'ORGANIZATION'), (u'Intelligence', u'ORGANIZATION'), (u'Analyst,', u'ORGANIZATION'), (u'Federal', u'ORGANIZATION'), (u'Reserve', u'ORGANIZATION')], [(u'Smartly', u'O')], [(u'Smartly', u'O')], [], [(u'Page1', u'O')], [], [(u'Michelle', u'PERSON'), (u'Pegler', u'PERSON')], []]
[u'Michelle Pegler', '', '', '', '', '', '', u'Michelle Pegler', '']


In [19]:
import requests
def basic(fname):
    url = "http://127.0.0.1:8000/resume/upload/"+fname

    files = {'file': open('/home/sumit/Desktop/project17/ResumeParser/Data/resume_samples/txts/samples/'+fname, 'rb')}
    r = requests.post(url, files=files)

    return r.json()['data']['basics']

In [23]:
name.append(basic(dir_list[0])['name']['firstName']+' '+basic(dir_list[0])['name']['surname'])
print name

[u'Michelle Pegler', '', '', '', '', '', '', u'Michelle Pegler', '', u'surname firstName', u'surname firstName', u'Michelle Pegler']
